# Called Kian? It's probably because of this guy.

The number of babies called Kian increased sharply after the band Westlife became popular. Is this relationship coincidental? Or is there some deeper meaning?

## Loading the data

Let's start by importing the packages we'll need:

In [ ]:
import cufflinks as cf
import numpy as np
import pandas as pd
import plot_helper as ph # Store custom plot code in a separate file for brevity
import plotly.offline as py

# Make sure we run cufflinks in offline mode
cf.go_offline()

### Irish baby name data

Birth name data for Ireland is available at the [Central Statistics Office](http://www.cso.ie/px/pxeirestat/Database/eirestat/Irish%20Babies%20Names/Irish%20Babies%20Names_statbank.asp?sp=Irish%20Babies%20Names&Planguage=0). Unfortunately, right now, the available data sets are incomplete:

- For the period 1964-2014, [data](http://www.cso.ie/px/pxeirestat/Statire/SelectVarVal/Define.asp?maintable=VSA05&PLanguage=0) is available for male names *only*.
  - Within this set, data for 1988 and 1997-2005 are currently unavailable (although I was told that this will be fixed soon).
- For the period 1998-2015, data is available for both [male](http://www.cso.ie/px/pxeirestat/Statire/SelectVarVal/Define.asp?maintable=VSA10&PLanguage=0) and [female](http://www.cso.ie/px/pxeirestat/Statire/SelectVarVal/Define.asp?maintable=VSA11&PLanguage=0) names.
  - Within this set, data for 1997-2005 is only *partially* available.
  - Not all names in a given year are represented.

Fortunately, the name Kian is represented in both datasets. All we need to do is combine them to form a complete (except for 1988) picture of popularity from 1964-2015:

In [ ]:
# Load the data from the raw CSV files and combine it into a unified data frame
vsa05 = pd.read_csv('VSA05.csv', index_col=0, skiprows=2, na_values='..')
vsa10 = pd.read_csv('VSA10.csv', index_col=0, skiprows=2, na_values='..')
baby_names = vsa05.combine_first(vsa10)

# Initially, the rows are the names and the columns are the dates, so swap these
baby_names = baby_names.T

# Set the index, so that each year in the period 1964-2015 is represented (some are missing)
baby_names.index = baby_names.index.map(pd.to_datetime)
baby_names = baby_names.reindex(pd.date_range(baby_names.index.min(), baby_names.index.max(), freq='AS'))
assert baby_names.shape[0] == 1 + (2015 - 1964)

### Birth rate data

The CSO also provide [data](http://www.cso.ie/px/pxeirestat/Statire/SelectVarVal/Define.asp?maintable=VSA18&PLanguage=0) for the total number of babies born in Ireland from 1864-2015:

In [ ]:
# Load the raw data
births = pd.read_csv('VSA18.csv', index_col=0, skiprows=2, na_values='..', skipfooter=2)

# Initially, the rows are the totals and the columns are the dates, so swap these
births = births.T

# Set the index, so that each year in the period 1864-2015 is represented (some are missing)
births.index = births.index.map(pd.to_datetime)
births = births.reindex(pd.date_range(births.index.min(), births.index.max(), freq='AS'))
assert births.shape[0] == 1 + (2015 - 1864)

### Westlife singles data

Finally, here's some data on Westlife singles that I scraped from [Wikipedia](https://en.wikipedia.org/wiki/Westlife_discography#Singles):

In [ ]:
westlife = pd.read_csv('westlife_singles.csv', index_col='Year', parse_dates=True).dropna(axis=0)

## Data cleaning and transformation

We've loaded the data, but it needs to be cleaned up before we can analyse it further as both the baby name and the Westlife singles data sets are missing data points in the time period we're interested in.

### Kian

Let's start by isolating the name Kian from the rest of the baby name data:

In [ ]:
kian = baby_names.Kian.copy()
kian.name = 'Babies called Kian'

Unfortunately, there are some gaps in the data:

In [ ]:
print "Number of missing points: %d" % kian.value_counts(dropna=False)[np.nan]

fig = ph.line_plot(kian)

fig.layout.update(
    title='Number of babies called Kian born in Ireland, 1964-2015',
    yaxis=dict(
        range=[0, 120],
        title='Number of babies'
    )
)

py.iplot(fig, show_link=False)

Prior to 2000, the number of Kians born each year is relatively small, so it's reasonable to assume that there were no, or almost no, Kians born in these years, i.e. we can replace these missing points with zeros.

However, there is also a missing data point at the end of the series, for the year 2015. In 2014, there were 72 Kians born and the years immediately prior to that show similar numbers. It therefore seems unlikely that that the popularity of the name collapsed completely in 2015. Data for 2016 and 2017 will verify this but, in the meantime, while we wait, let's just leave 2015 as a missing data point. This restricts our analysis to the period 1964-2014.

In [ ]:
# Fill missing values with zeros
kian.fillna(0, inplace=True)

# Drop 2015 from the set
kian.drop(pd.Timestamp('2015'), inplace=True)

# Plot the results
fig = ph.line_plot(kian)

fig.layout.update(
    title='Number of babies called Kian born in Ireland, 1964-2014',
    yaxis=dict(
        range=[0, 120],
        title='Number of babies'
    )
)

py.iplot(fig, show_link=False)

While the raw data for Kian gives an impression of its popularity, it doesn't account for the variation in the number of babies born from year to year. For instance, if the number of Kians born in a given year is smaller than in another, it might be because of some underlying trend, but it could equally be due to differences in the total numbers of babies born in both years.

A better way to think about popularity, then, is the *percentage* of babies named Kian in a given year:

In [ ]:
# Compute the percentage of babies called Kian in a given year
kian_pc = kian / births.Male.ix['1964':'2014']
kian_pc.name = kian.name

# Plot the results
fig = ph.line_plot(kian_pc)

fig.layout.update(
    title='Percentage of babies called Kian born in Ireland, 1964-2014',
    yaxis=dict(
        hoverformat='.2%',
        range=[0, 0.004],
        tickformat='.2%',
        title='Percentage of babies'
    )
)

py.iplot(fig, show_link=False)

### Westlife

Let's start by extracting the number of Westlife singles released each year:

In [ ]:
singles = westlife.dropna(axis=0).index.value_counts().reindex(kian.index)
singles.name = 'Westlife singles'

There are missing points for years where Westlife didn't release any singles:

In [ ]:
print "Number of missing data points: %d" % singles.value_counts(dropna=False)[np.nan]

fig = ph.line_plot(singles)

fig.data[0].line.update(color=cf.get_colorscale(cf.get_config_file()['colorscale'])[1][1])

fig.layout.update(
    title='Westlife single releases, 1964-2014',
    yaxis=dict(
        range=[0, 5],
        title='Number of singles'
    )
)

py.iplot(fig, show_link=False)

This time we know that, wherever we have no data, it's because Westlife released no singles, so we can safely fill in any missing data points with zeros:

In [ ]:
singles.fillna(0, inplace=True)

fig = ph.line_plot(singles)

fig.data[0].line.update(color=cf.get_colorscale(cf.get_config_file()['colorscale'])[1][1])

fig.layout.update(
    title='Westlife single releases, 1964-2014',
    yaxis=dict(
        range=[0, 5],
        title='Number of singles'
    )
)

py.iplot(fig, show_link=False)

## Visualising the relationship

Now that we've got clean data sets, we can start to look at the relationship between them:

In [ ]:
data = pd.concat([kian_pc, singles], axis=1)
fig = ph.line_plot(data, secondary_y=data.columns[1])

fig.layout.update(
    title='Popularity of the name Kian, 1964-2014',
    yaxis1=dict(
        dtick=0.0006,
        range=[0, 0.0036],
        title='Percentage of births'
    ),
    yaxis2=dict(
        range=[0, 6],
        title='Number of singles'
    )
)

py.iplot(fig, show_link=False)

Looking at both data sets together, it's clear that the name Kian became popular shortly after Westlife did. Specifically, it looks like the popularity of Kian lags the birth of Westlife by about two years. We can see this even more clearly if we lag the singles data by two years and use a scatter plot matrix:

In [ ]:
fig = ph.scatter_plot_matrix(pd.concat([kian_pc, singles.shift(2)], axis=1),
                             y1_title='Percentage of births', y2_title='Number of singles (lagged)')

fig.layout.update(
    title='Popularity of the name Kian, 1964-2014',
    yaxis1=dict(
        dtick=0.0006,
        range=[0, 0.0036]
    ),
    yaxis6=dict(
        range=[0, 6]
    )
)

py.iplot(fig, show_link=False)

While it's tempting to say that Westlife caused Kian to become popular, we have to be careful not to wander into statistical traps. [Correlation does not imply causation.](https://en.wikipedia.org/wiki/Correlation_does_not_imply_causation) However, we can get a little more specific.

## Granger causality test

The [Granger causality test](https://en.wikipedia.org/wiki/Granger_causality) is a statistical test that can be used to determine whether one time series is a good predictor of another. More specifically, if we test to see if A is *Granger-caused* by B, and the test passes, we can then state that the past values of both A and B are a better predictor of A than the past values of A alone. This is not as strong a statement as saying that A is *caused* by B, but it does go further than simply stating that A and B are correlated.

Let's start by importing the test tools:

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.tsatools import lagmat

The Granger causality test works by building two [vector regression](https://en.wikipedia.org/wiki/Vector_autoregression) models of the target time series. The first model is constructed from the past values of the target time series alone, while the second is built using the past values of the target series and the past values of another "candidate" series, for which we are testing the Granger causality hypothesis. Various hypothesis tests (e.g. $F$ test, $\chi^2$ test, likelihood ratio test) can be performed to determine if a Granger causal relationship exists. Generally, the relationship is declared to exist if we reject the null hypothesis at a certain significance level (e.g. $p<0.05$).

In cases where there is a [structural break](https://en.wikipedia.org/wiki/Structural_break) in the data, we can either introduce [dummy variables](#Predicting-popularity) to the regression or just check for the presence of a causal relationship on either side of the break separately. In our data, there is a structural break around 1999, when Westlife first began to release singles. Before this point, the Westlife single series has all-zero values, so it doesn't make sense to test for Granger causality here (an all zero vector has no predictive power in a regression), but we can do the check from 1999 onwards, up to a maximum of four lags.

In [ ]:
results = grangercausalitytests(data.ix['1999':], 4)

The test results show that a Granger causal relationship exists for lags of one and two years, although not for higher lags, at significance level of $p<0.05$. This effectively confirms our earlier suspicion: that Westlife singles have significant predictive power in forecasting the popularity of the name Kian.

## Predicting popularity

Now that we've confirmed that there's a Granger causal relationship in the data, we can exploit it to predict the popularity of Kian over time.

First, we'll need to import the `LinearRegression` class from `sklearn`:

In [ ]:
from sklearn.linear_model import LinearRegression

Earlier, we did a Granger causality test on the right hand side of the structural break in the Westlife singles data. This was mainly because the `grangercausalitytests` function in `statsmodels` is [limited](http://statsmodels.sourceforge.net/0.6.0/generated/statsmodels.tsa.stattools.grangercausalitytests.html) to testing for causality in two time series only, so it wasn't possible to include a dummy variable in the regression. However, using `sklearn`, this isn't a problem.

Let's create a dummy variable to account for the fact that Westlife have only been in existence since 1999:

In [ ]:
# Create the dummy variable
dummy = pd.Series([0] * (1999 - 1964) + [1] * (2015 - 1999), index=singles.index)

# Make a quick plot, so we can see what it looks like
fig = ph.line_plot(dummy)

fig.data[0].line.update(color='grey')

fig.layout.update(
    height=400,
    title='A dummy variable indicating the pre and post Westlife eras',
    yaxis=dict(range=[0,1])
)

py.iplot(fig, show_link=False)

Next, we'll create a lagged matrix (of two years) from the popularity data, the Westlife singles data and the dummy variable data and use this to build a regression model of the popularity of Kian over time:

In [ ]:
# Build the model
X = lagmat(pd.concat([kian_pc, singles, dummy], axis=1).values, 2)
y = kian_pc.values

clf = LinearRegression(fit_intercept=False)
clf.fit(X, y)

predicted = pd.Series(clf.predict(X), index=kian_pc.index)
predicted.name = 'Model prediction'

# Plot the results
fig = ph.line_plot(pd.concat([kian_pc, predicted], axis=1))

fig.layout.update(
    annotations=ph.annotate_metrics(y, predicted.values, 0.025, 1.025),
    title='Predicting the popularity of the name Kian in Ireland, 1964-2014',
    yaxis=dict(
        hoverformat='.2%',
        range=[0, 0.005],
        tickformat='.2%',
        title='Percentage of births'
    )
)

py.iplot(fig, show_link=False)

As you can see, the model predicts the popularity of Kian quite well, with a coefficient of determination value of $r^2 = 0.959$. However, if we exclude all the past popularity data from the predictor variables, the results are even more remarkable:

In [ ]:
# Build the model
X = lagmat(pd.concat([singles, dummy], axis=1).values, 2)
y = kian_pc.values

clf = LinearRegression(fit_intercept=False)
clf.fit(X, y)

predicted = pd.Series(clf.predict(X), index=kian_pc.index)
predicted.name = 'Model prediction (Westlife singles only)'

# Plot the results
fig = ph.line_plot(pd.concat([kian_pc, predicted], axis=1))

fig.layout.update(
    annotations=ph.annotate_metrics(y, predicted.values, 0.025, 1.025),
    title='Predicting the popularity of the name Kian in Ireland, 1964-2014',
    yaxis=dict(
        hoverformat='.2%',
        range=[0, 0.005],
        tickformat='.2%',
        title='Percentage of births'
    )
)

py.iplot(fig, show_link=False)

While this model is a little less accurate than the first, it doesn't rely on the past popularity of the name Kian when predicting its future popularity. Instead, it accurately predicts the popularity of the name based on the number of Westlife singles released in a given year alone.


## Conclusion

Using the Granger causality test and a standard vector regression, it's possible to build an accurate model that explains the current popularity of the name Kian using the number of Westlife singles released in the past two years.

The popularity of the names of other Westlife members (Bryan/Brian, Mark, Nicky and Shane) are not as straighforward to model, as they (or their most common variants) were never as unpopular as the name Kian prior to Westlife releasing their first single.

As Westlife last released a single in 2011, the current model will likely fail in 2015 and beyond, although it will be interesting to see whether future deviations from its predictions can be explained by Kian Egan's [solo releases](https://en.wikipedia.org/wiki/Kian_Egan#Discography), or taking into account the chart positions of singles, which I haven't done here but are a more intuitive indicator of popularity than the raw number of singles released in a given year.